# Use SPSS to predict customer churn with `ibm-watson-machine-learning`

This notebook contains steps to deploy sample SPSS stream, and start scoring new data. 

Some familiarity with bash is helpful. This notebook uses Python 3.10.

You will use a data set, **Telco Customer Churn**, which details anonymous customer data from a telecommunication company. Use the details of this data set to predict customer churn which is very critical to business as it's easier to retain existing customers rather than acquiring new ones.

## Learning goals

The learning goals of this notebook are:

-  Working with the Watson Machine Learning instance
-  Online deployment of SPSS model
-  Scoring data using deployed model


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Model upload](#upload) 
3. [Web service creation](#deploy)
4. [Scoring](#score)
5. [Clean up](#cleanup)
6. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [2]:
wml_credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.7'
}

Alternatively you can use `username` and `password` to authenticate WML services.

```
wml_credentials = {
    "username": ***,
    "password": ***,
    "url": ***,
    "instance_id": 'openshift',
    "version": '4.7'
}

```

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="https://ibm.github.io/watson-machine-learning-sdk//" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.6/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Upload model

In this section you will learn how to upload the model to the Cloud.


**Action**: Download sample SPSS model from git project using wget.

In [6]:
import os
from wget import download

sample_dir = 'spss_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename=os.path.join(sample_dir, 'customer-satisfaction-prediction.str')
if not os.path.isfile(filename):
    filename = download('https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.6/models/spss/customer_satisfaction/model/customer-satisfaction-prediction.str',\
                             out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


Store SPSS sample model in your Watson Machine Learning instance.

In [ ]:
client.software_specifications.list()

In [7]:
sw_spec_uid = client.software_specifications.get_uid_by_name("spss-modeler_18.2")

model_meta_props = {
    client.repository.ModelMetaNames.NAME: "SPSS customer satisfaction model",
    client.repository.ModelMetaNames.TYPE: "spss-modeler_18.2",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
}

model_details = client.repository.store_model(filename, model_meta_props)

**Note:** You can see that model is successfully stored in Watson Machine Learning Service.

In [ ]:
client.repository.list_models()

<a id="deploy"></a>
## 3. Create online deployment
You can use commands bellow to create online deployment for stored model (web service).

In [8]:
model_uid = client.repository.get_model_uid(model_details)

deployment = client.deployments.create(
    artifact_uid=model_uid,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "SPSS deployment",
        client.deployments.ConfigurationMetaNames.ONLINE:{}}
)



#######################################################################################

Synchronous deployment creation for uid: '83b07fb3-32df-47c7-8cc1-71081c77e1a2' started

#######################################################################################


initializing.........................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='3d009190-2cb0-46b7-9744-25e1d8429d71'
------------------------------------------------------------------------------------------------




<a id="score"></a>
## 4. Scoring

You can send new scoring records to web-service deployment using `score` method.

In [9]:
deployment_id = client.deployments.get_id(deployment)

scoring_data = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["customerID","gender","SeniorCitizen","Partner","Dependents","tenure","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod","MonthlyCharges","TotalCharges","Churn","SampleWeight"],
        "values":[["3638-WEABW","Female",0,"Yes","No",58,"Yes","Yes","DSL","No","Yes","No","Yes","No","No","Two year","Yes","Credit card (automatic)",59.9,3505.1,"No",2.768]]
    }]
}

predictions = client.deployments.score(deployment_id, scoring_data)
print(predictions)

{'predictions': [{'fields': ['customerID', 'Churn', 'Predicted Churn', 'Probability of Churn'], 'values': [['3638-WEABW', 'No', 'No', 0.0526309571556145]]}]}


As we can see this sample telco customer is satisfied ("Predicted Churn", "No").

<a id="cleanup"></a>
## 5. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.6/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for SPSS model deployment and scoring. 
 
 Check out our [Online Documentation](https://www.ibm.com/docs/en/cloud-paks/cp-data/4.5.x?topic=deploying-managing-models-functions) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Lukasz Cmielowski, PhD,** is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2020, 2021, 2022, 2023 IBM. This notebook and its source code are released under the terms of the MIT License.